In [75]:
import re
import bs4
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Himank
[nltk_data]     K\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
df = pd.read_csv('all_kindle_review.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [13]:
df = df[['reviewText', 'rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [14]:
df.shape

(12000, 2)

In [15]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [16]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [17]:
df['rating'] = df['rating'].apply(lambda x: 0 if x<3 else 1)
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [18]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [19]:
df['reviewText'] = df['reviewText'].str.lower()
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [ ]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

In [76]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    text = ' '.join([lemmatizer.lemmatize(word, 'n') for word in text.split()])
    text = ' '.join([lemmatizer.lemmatize(word, 'v') for word in text.split()])
    text = ' '.join([lemmatizer.lemmatize(word, 'a') for word in text.split()])
    text = ' '.join([lemmatizer.lemmatize(word, 'r') for word in text.split()])
    return ' '.join([lemmatizer.lemmatize(word, 's') for word in text.split()])

df['reviewText'] = df['reviewText'].apply(lambda x: lemmatize_words(x))

In [20]:
df = pd.read_csv('cleaned_data.csv')
df.head()

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read i want put i read one sitting...,1
2,i start saying first four book i expecting 34 ...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,i expect type book library pleased find price ...,1


In [77]:
X = df['reviewText'].values
y = df['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [78]:
words = [i.split() for i in X_train]

In [141]:
model = Word2Vec(words)
model.corpus_count, model.epochs

(9600, 5)

In [80]:
def avg_word2vec(doc):
    valid_words = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    return np.mean(valid_words, axis=0) if valid_words else np.zeros(model.vector_size)

def vectorize(column):
    col_vec = [i.split() for i in column]
    for i in tqdm(range(len(col_vec))):
        col_vec[i] = avg_word2vec(col_vec[i])
    return np.array(col_vec)

In [81]:
X_train_vec = vectorize(X_train)
X_test_vec = vectorize(X_test)

100%|██████████| 2400/2400 [00:01<00:00, 1269.45it/s]


In [82]:
pd.DataFrame(X_train).head()

,0
0,i look forward book then come doubl space ever...
1,we alreadi own book my spous forget alreadi pa...
2,cool forget i request rate come make mine unre...
3,this short short stori it basic scene parti on...
4,all secret servic agent secrest even long serv...


In [93]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
nn_model = Sequential([
    Dense(128, activation=tf.keras.activations.relu, input_shape=(X_train_vec.shape[1],)),
    Dense(64, activation=tf.keras.activations.relu),
    Dense(1, activation=tf.keras.activations.sigmoid)
])

d:\ML\Kindle-Sentiment-Analysis\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [111]:
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [ ]:
history = nn_model.fit(X_train_vec, y_train,
                    validation_data=(X_test_vec, y_test),
                    epochs=100,
                    callbacks=[early_stopping_callback])

Epoch 1/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9399 - loss: 0.1482 - val_accuracy: 0.7292 - val_loss: 1.0392
Epoch 2/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9348 - loss: 0.1537 - val_accuracy: 0.7500 - val_loss: 1.0198
Epoch 3/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9394 - loss: 0.1541 - val_accuracy: 0.7483 - val_loss: 1.0418
Epoch 4/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9379 - loss: 0.1546 - val_accuracy: 0.7475 - val_loss: 1.0469
Epoch 5/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9370 - loss: 0.1512 - val_accuracy: 0.7450 - val_loss: 1.0356
Epoch 6/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9329 - loss: 0.1539 - val_accuracy: 0.7563 - val_loss: 1.0902
Epoch 7/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9348 - loss: 0.1499 - val_accuracy: 0.7471 - val_loss: 1.0383
Epoch 8/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9361 - loss: 0.1524 - val_accu

In [ ]:
y_pred = nn_model.predict(X_test_vec).flatten('C')
y_pred = np.where(y_pred > 0.5, 1, 0)

75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step


In [137]:
accuracy_score(y_test, y_pred)

0.7383333333333333

In [140]:
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train_vec, y_train)
y_pred = classifier.predict(X_test_vec)

In [142]:
classifier.predict(vectorize(['This book is bad']))

100%|██████████| 1/1 [00:00<00:00, 999.60it/s]


array([0], dtype=int64)

In [143]:
import pickle

In [144]:
with open('artifacts/classifier.pkl', 'wb+') as file:
    pickle.dump(classifier, file)